<a href="https://colab.research.google.com/github/BaronVonBussin/NewTransit/blob/main/voodoo_rails_20250222.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd

def process_rail_count(file_path, output_path):
    # Load data
    df = pd.read_csv(file_path)
    df.sort_values(by='Date', ascending=True, inplace=True)

    # Initialize new fields
    df['rail_count'] = 0
    df['rail_high'] = None
    df['rail_low'] = None
    df['rail_range'] = None
    df['rail_overlap_at_open'] = False
    df['rail_distance'] = None
    df['rail_distance_percent'] = None
    df['rail_direction'] = None
    df['rail_crossover_at_close'] = False

    # Iterate through each row as the current row
    for current_idx in range(len(df)):
        # Initialize reference range using the current row's high and low
        ref_high = df.loc[current_idx, 'H']
        ref_low = df.loc[current_idx, 'L']
        rail_count = 1  # Start with the current row counted

        # Get the current open for later calculations
        current_open = df.loc[current_idx, 'O']
        current_high = df.loc[current_idx, 'H']
        current_low = df.loc[current_idx, 'L']

        # Check previous rows in reverse order
        for past_idx in range(current_idx - 1, -1, -1):
            past_high = df.loc[past_idx, 'H']
            past_low = df.loc[past_idx, 'L']

            # Stop condition: past row is completely outside reference range
            if past_low > ref_high or past_high < ref_low:
                break

            # Expand reference range
            ref_high = min(ref_high, past_high)
            ref_low = max(ref_low, past_low)
            rail_count += 1

        # Store basic rail results
        df.loc[current_idx, 'rail_count'] = rail_count
        df.loc[current_idx, 'rail_high'] = ref_high
        df.loc[current_idx, 'rail_low'] = ref_low
        df.loc[current_idx, 'rail_range'] = ref_high - ref_low

        # Skip first row for relative calculations
        if current_idx > 0:
            # Get previous rail values
            prev_rail_high = df.loc[current_idx - 1, 'rail_high']
            prev_rail_low = df.loc[current_idx - 1, 'rail_low']

            # 1. Check if open is within previous rail
            df.loc[current_idx, 'rail_overlap_at_open'] = (
                current_open >= prev_rail_low and
                current_open <= prev_rail_high
            )

            # 2. Calculate distance if outside rail
            if not df.loc[current_idx, 'rail_overlap_at_open']:
                # Find closest rail boundary
                distance_to_high = current_open - prev_rail_high
                distance_to_low = prev_rail_low - current_open

                if abs(distance_to_high) < abs(distance_to_low):
                    rail_distance = distance_to_high
                else:
                    rail_distance = distance_to_low

                df.loc[current_idx, 'rail_distance'] = abs(rail_distance)
                df.loc[current_idx, 'rail_distance_percent'] = (
                    abs(rail_distance) / current_open
                )

            # 3. Determine rail direction
            if current_open > prev_rail_high:
                df.loc[current_idx, 'rail_direction'] = 'D'
            elif current_open < prev_rail_low:
                df.loc[current_idx, 'rail_direction'] = 'U'

            # 4. Check if bar touches rail
            df.loc[current_idx, 'rail_crossover_at_close'] = (
                (current_high >= prev_rail_low and current_high <= prev_rail_high) or
                (current_low >= prev_rail_low and current_low <= prev_rail_high) or
                (current_low <= prev_rail_low and current_high >= prev_rail_high)
            )

    # Save output file
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df.to_csv(output_path, index=False)

def main():
    input_folder = '/content/input/'
    output_folder = '/content/export/'

    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Process each file in the input directory
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.csv'):
            file_path = os.path.join(input_folder, file_name)
            output_file = file_name.replace('.csv', '_export.csv')
            output_path = os.path.join(output_folder, output_file)

            print(f'Processing: {file_name}')
            process_rail_count(file_path, output_path)
            print(f'Exported: {output_file}')

if __name__ == "__main__":
    main()